In [1]:
import os 
from dotenv import load_dotenv,find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

### Completion Model

In [2]:
from langchain_openai import OpenAI

llmModel = OpenAI()

### Chat-Completion Model

In [3]:
from langchain_openai import ChatOpenAI

chatModel = ChatOpenAI(model = "gpt-3.5-turbo-0125")

### Prompts and PromptTemplates

In [4]:
# This is for completion Model
from langchain_core.prompts import PromptTemplate
prompt_template = PromptTemplate.from_template(
    "Tell me a {adjective} story about {topic}"
)

llmModelPrompt = prompt_template.format(
    adjective = "curious",
    topic = "The Kennedy family"
)

res = llmModel.invoke(llmModelPrompt)
print(res)



One of the most well-known and controversial families in American history, the Kennedys have a long history of political power and personal tragedies. However, one lesser-known aspect of the Kennedy family is their connection to the world of competitive sailing.

In 1932, Joseph P. Kennedy Sr., the patriarch of the Kennedy family, purchased a 50-foot sailboat named Victura for his children to learn how to sail. The boat quickly became a favorite pastime for the Kennedy children, particularly John F. Kennedy, who became an accomplished sailor and even skippered Victura in several races.

But it wasn't until 1962 that the Kennedy family's love for sailing took a strange turn. In the midst of the Cold War, tensions were high between the United States and the Soviet Union. In an effort to ease the tension, President Kennedy proposed a series of friendly sporting competitions between the two countries, including a sailing race.

The Soviets, eager to prove their superiority, accepted the 

### Prompt Template for ChatCompletionModel

In [5]:
from langchain_core.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
        ("system","You are an {profession} expert on {topic}"),
        ("human","Hello, Mr. {profession}, can you please answer a question?"),
        ("ai","Sure!"),
        ("human","{user_input}"),
    ]
)

messages = chat_template.format_messages(
    profession = "Historian",
    topic = "The Kennedy Family",
    user_input = "How many grand children had Joseph P. Kennedy?"
)

response = chatModel.invoke(messages)

print(response)


content='Joseph P. Kennedy, Sr., the patriarch of the Kennedy family, had a total of 34 grandchildren.' response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 55, 'total_tokens': 77, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-d954d213-6007-466e-bebb-cff3dce7a3af-0' usage_metadata={'input_tokens': 55, 'output_tokens': 22, 'total_tokens': 77}


# Few Shot Prompting

In [11]:
from langchain_core.prompts import FewShotChatMessagePromptTemplate

examples = [
    {"input":"hi!","output":"hola!"},
    {"input":"bye!","output":"adios!"}
]

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human","{input}"),
        ("ai","{output}"),
    ]
)

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt = example_prompt,
    examples = examples
)

final_prompt = ChatPromptTemplate.from_messages([
    ("system","You are an English-spanish translator"),
    few_shot_prompt,
    ("human","{input}"),
    ]
)

print(few_shot_prompt)

examples=[{'input': 'hi!', 'output': 'hola!'}, {'input': 'bye!', 'output': 'adios!'}] example_prompt=ChatPromptTemplate(input_variables=['input', 'output'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')), AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=['output'], template='{output}'))])


### Chains

In [13]:
from langchain_core.prompts import FewShotChatMessagePromptTemplate

examples = [
    {"input":"hi!","output":"hola!"},
    {"input":"bye!","output":"adios!"}
]

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human","{input}"),
        ("ai","{output}"),
    ]
)

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt = example_prompt,
    examples = examples
)

final_prompt = ChatPromptTemplate.from_messages([
    ("system","You are an English-spanish translator"),
    few_shot_prompt,
    ("human","{input}"),
    ]
)

chain = final_prompt | chatModel 

res = chain.invoke({"input": "How are you?"})
print(res.content)

¿Cómo estás?


### Output Parsers

In [16]:
from langchain_core.prompts import PromptTemplate
from langchain.output_parsers.json import SimpleJsonOutputParser

json_prompt = PromptTemplate.from_template(
    "Return a JSON Object with an 'answer' key that answers the following question: {question}"
)

json_parser = SimpleJsonOutputParser()

json_chain = json_prompt | llmModel | json_parser

json_chain.invoke({"question": "What is the population of India in present?"})

{'answer': 'As of 2021, the population of India is approximately 1.38 billion.'}

### Optionally, you can use pydantic to define a custom output format

In [17]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel,Field 



In [18]:
class Joke(BaseModel):
    setup: str = Field(description="Question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")

In [20]:
# Define the parser referring the pydantic object
parser = JsonOutputParser(pydantic_object=Joke)

prompt = PromptTemplate(
    template = "Answer the user query. \n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | chatModel | parser 

chain.invoke({"query": "Tell me a joke."})

{'setup': "Why couldn't the bicycle stand up by itself?",
 'punchline': 'It was two tired!'}